In [28]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import urllib.parse 

In [29]:
def scrape_jobs(position,location):
    position = urllib.parse.quote_plus(position)
    location = urllib.parse.quote_plus(location)
    sequence = 1
    data = []

    while True:
        start_page = 1 + ((sequence - 1) // 10) * 10
        template = 'https://www.timesjobs.com/candidate/job-search.html?from=submit&luceneResultSize=100&txtKeywords=0DQT0{}0DQT0&postWeek=60&searchType=personalizedSearch&actualTxtKeywords={}&searchBy=0&rdoOperator=OR&txtLocation={}&pDate=I&sequence={}&startPage={}'
        url = template.format(position,position, location, sequence, start_page)
        
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'lxml')
        jobs = soup.find_all('li',class_='clearfix job-bx wht-shd-bx')
        print('jobs found', len(jobs),'url',url)

        if not jobs:
            break
        sequence += 1

        for job in jobs:
            try:
                job_role = job.find('h2').text.strip()
            except AttributeError:
                job_role = ''
            try:
                company_name = job.find('h3',class_='joblist-comp-name').text.strip()
            except AttributeError:
                company_name = ''
            try:
                location_company = job.find('span').text.strip()
            except AttributeError:
                location_company = ''
            try:
                posted_date = (job.find('span',class_='sim-posted').span.text).split('Posted')[-1].strip()
            except AttributeError:
                posted_date = ''
            try:
                skill = job.find('span',class_='srp-skills').text.strip()
            except AttributeError:
                skill = ''
            try:
                job_desc = job.find('label', string='Job Description:').find_next_sibling('a').previous_sibling.strip()
            except AttributeError:
                job_desc = '' 
            try:
                years_exp = job.select_one('ul.top-jd-dtl li:has(i.material-icons:contains("card_travel"))').text.replace('card_travel', '').strip()
            except AttributeError:
                experience = ''
            try:
                more_detail_link = job.header.h2.a['href']
            except AttributeError:
                more_detail_link = ''
            
            extract_date = datetime.today().strftime('%Y-%m-%d')
            
            data.append({'Role':job_role, 'Company Name':company_name, 'Location':location_company, 'Posted Date':posted_date,'Extracted Date':extract_date,'Key Skill': skill,'Experience':years_exp,'Job Description':job_desc, 'More Detail':more_detail_link})
          
    df = pd.DataFrame(data)
    
    df.to_csv(f'list_of_{position}_timesjobs.csv', index=False, encoding='utf-8')

In [30]:
if __name__ == '__main__':
    position = 'data analyst'
    location = ''

    scrape_jobs(position,location)

jobs found 100 url https://www.timesjobs.com/candidate/job-search.html?from=submit&luceneResultSize=100&txtKeywords=0DQT0data+analyst0DQT0&postWeek=60&searchType=personalizedSearch&actualTxtKeywords=data+analyst&searchBy=0&rdoOperator=OR&txtLocation=&pDate=I&sequence=1&startPage=1
jobs found 100 url https://www.timesjobs.com/candidate/job-search.html?from=submit&luceneResultSize=100&txtKeywords=0DQT0data+analyst0DQT0&postWeek=60&searchType=personalizedSearch&actualTxtKeywords=data+analyst&searchBy=0&rdoOperator=OR&txtLocation=&pDate=I&sequence=2&startPage=1
jobs found 82 url https://www.timesjobs.com/candidate/job-search.html?from=submit&luceneResultSize=100&txtKeywords=0DQT0data+analyst0DQT0&postWeek=60&searchType=personalizedSearch&actualTxtKeywords=data+analyst&searchBy=0&rdoOperator=OR&txtLocation=&pDate=I&sequence=3&startPage=1
jobs found 0 url https://www.timesjobs.com/candidate/job-search.html?from=submit&luceneResultSize=100&txtKeywords=0DQT0data+analyst0DQT0&postWeek=60&search

In [31]:
timejobs_dataset = pd.read_csv(f'list_of_data+analyst_timesjobs.csv')

In [32]:
timejobs_dataset

,Role,Company Name,Location,Posted Date,Extracted Date,Key Skill,Experience,Job Description,More Detail
0,Data Analyst - Associate Data Analyst / Data...,Information Services Group,Bengaluru / Bangalore,few days ago,2024-02-19,"data analysis , python , excel , d...",2 - 3 yrs,Data Analyst - Associate Data Analyst / Data...,https://www.timesjobs.com/job-detail/data-anal...
1,Data Analyst - Associate Data Analyst / Data...,Information Services Group,Bengaluru / Bangalore,few days ago,2024-02-19,"data analysis , python , excel , d...",2 - 3 yrs,Data Analyst - Associate Data Analyst / Data...,https://www.timesjobs.com/job-detail/data-anal...
2,Data Analysts,Zoxima Solutions Pvt. Ltd.,NaN,few days ago,2024-02-19,"reports , Data Analysts , MySQL",3 - 8 yrs,Experience 3-8 YearsJOB INFORMATIONVery good ...,https://www.timesjobs.com/job-detail/data-anal...
3,Data Analyst,the modern dimension,"Delhi, Delhi/NCR, Bengaluru / Bangalore",4 days ago,2024-02-19,"oracle , sas , bi , dashboards , data ...",3 - 6 yrs,Responsibilities Building advanced workflows u...,https://www.timesjobs.com/job-detail/data-anal...
4,Data Analyst,ADROIT LEARNING AND MANPOWER PVT LTD,Ahmedabad,5 days ago,2024-02-19,"data analysis , dashboards , data analyst ...",7 - 10 yrs,You will work on and contribute into building ...,https://www.timesjobs.com/job-detail/data-anal...
...,...,...,...,...,...,...,...,...,...
277,Data Analysis Internship in Pune,Maxgen Technologies\n (More Jobs),(More Jobs),few days ago,2024-02-19,.,0 - 1 yrs,Maxgen Technologies Pvt Ltd is an it company b...,https://www.timesjobs.com/job-detail/data-anal...
278,Full Stack Web Developer,GENIQUE MANAGEMENT SERVICES PRIVATE LIMITED\n ...,(More Jobs),few days ago,2024-02-19,"full stack , java , Html , css , .net ...",6 - 11 yrs,Hi We are hiring for various IT positions .Sof...,https://www.timesjobs.com/job-detail/full-stac...
279,"audit , financial accounting",Laxmi Infotech\n (More Jobs),(More Jobs),a month ago,2024-02-19,"Banking , accounts , Sales Tax , Income ...",1 - 6 yrs,Sales Invoice Purchase Invoice entriesMust be ...,https://www.timesjobs.com/job-detail/audit-fin...
280,Account Executive,Laxmi Infotech\n (More Jobs),(More Jobs),4 days ago,2024-02-19,"Finance , Banking , accounts , Sales Tax...",0 - 3 yrs,Sales Invoice Purchase Invoice entriesMust be ...,https://www.timesjobs.com/job-detail/account-e...
